## Begin the ETL 
First of all, we'll begin with the extraction of dataframe from csv file
This project require some module we have to use with python:
- python v3.10.x and higher
- pandas module
- dotenv for the environment variable management

In [3]:
import pandas as pd

demographic_data = pd.read_csv('Demographic_Data.csv')
geographic_data = pd.read_csv('Geographic_Data.csv')


## Merge the two csv files
We merge the tables to collect one table of the same view 

In [4]:
merged_data = geographic_data.merge(demographic_data,'inner','Location')

merged_data

,Location,Altitude (m),Proximity to Industry (km),Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher)
0,Los Angeles,89,5.0,10039107,3276,89,60000,32
1,Paris,35,3.0,2140526,21383,98,45000,29
2,Tokyo,40,2.0,13929286,6169,100,55000,37
3,Antananarivo,1276,0.5,1391433,3097,69,1000,10
4,Nairobi,1795,1.0,4397073,6000,61,1500,12
5,Lima,1540,1.5,9674755,3220,81,4500,15


## Creating a DataFrame for Longitude and Latitude
Our current dataset does not include longitude and latitude data, so we need to create a new DataFrame to store this information.

In [8]:
# Create a list of dictionaries for each location
locations = [
    {'Location': 'Antananarivo', 'lon': 47.5256, 'lat': -18.91},
    {'Location': 'Los Angeles', 'lon': -118.242766, 'lat': 34.0536909},
    {'Location': 'Paris', 'lon': 2.320041, 'lat': 48.8588897},
    {'Location': 'Nairobi', 'lon': 36.8172, 'lat': -1.2833},
    {'Location': 'Lima', 'lon': -77.0365256, 'lat': -12.0621065},
    {'Location': 'Tokyo', 'lon': 139.762221, 'lat': 35.6821936}
]

# Directly create a DataFrame from the list of dictionaries
lon_and_lat_df = pd.DataFrame(locations)

lon_and_lat_df

,Location,lon,lat
0,Antananarivo,47.525600,-18.910000
1,Los Angeles,-118.242766,34.053691
2,Paris,2.320041,48.858890
3,Nairobi,36.817200,-1.283300
4,Lima,-77.036526,-12.062107
5,Tokyo,139.762221,35.682194


## Merging Geographic Coordinates with the Existing DataFrame
In this section, we will merge a DataFrame containing geographic coordinates (longitude and latitude) with our existing DataFrame. This process will enhance our dataset by associating each location with its corresponding coordinates

In [9]:
new_merged_data = merged_data.merge(lon_and_lat_df,'inner','Location')
new_merged_data

,Location,Altitude (m),Proximity to Industry (km),Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher),lon,lat
0,Los Angeles,89,5.0,10039107,3276,89,60000,32,-118.242766,34.053691
1,Paris,35,3.0,2140526,21383,98,45000,29,2.320041,48.858890
2,Tokyo,40,2.0,13929286,6169,100,55000,37,139.762221,35.682194
3,Antananarivo,1276,0.5,1391433,3097,69,1000,10,47.525600,-18.910000
4,Nairobi,1795,1.0,4397073,6000,61,1500,12,36.817200,-1.283300
5,Lima,1540,1.5,9674755,3220,81,4500,15,-77.036526,-12.062107


## Load data from Openweathermap and merge with the existent dataframe

In [10]:
import requests as rq
import os
from dotenv import load_dotenv
load_dotenv()

# Variable environment for API key
api_key = os.getenv('API_KEY')
# Base URL for the API
base_url = 'http://api.openweathermap.org/data/2.5/air_pollution'

# Get the aqi data from api
def get_aqi_data(row):
    lon = row['lon']
    lat = row['lat']

    response = rq.get(f'{base_url}?lat={lat}&lon={lon}&appid={api_key}')
    data = response.json()
    return data['list'][0]['main']['aqi']


# Get polluting gases data

def get_components_data(row):
    lon = row['lon']
    lat = row['lat']
    response = rq.get(f'{base_url}?lat={lat}&lon={lon}&appid={api_key}')
    data = response.json()
    return data['list'][0]['components']
# Add a new column 'AQI' to the DataFrame by applying the get_aqi_data function
new_merged_data['AQI'] = new_merged_data.apply(get_aqi_data, axis=1)

# Add new columns for pollutants by applying the get_components_data function
components_data = new_merged_data.apply(get_components_data, axis=1)

# Convert the list of dictionaries to a DataFrame
components_df = pd.DataFrame(components_data.tolist())

# Concatenate the components DataFrame with the existing DataFrame
merged_df = pd.concat([new_merged_data, components_df], axis=1)

merged_df



,Location,Altitude (m),Proximity to Industry (km),Population,Density (people/km²),Urbanization (%),Average Income (USD),Education Level (% with Bachelor's or higher),lon,lat,AQI,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,Los Angeles,89,5.0,10039107,3276,89,60000,32,-118.242766,34.053691,3,500.68,8.94,86.37,0.01,12.04,11.03,17.69,2.63
1,Paris,35,3.0,2140526,21383,98,45000,29,2.320041,48.858890,2,303.75,4.14,16.97,41.49,2.33,11.47,13.05,1.20
2,Tokyo,40,2.0,13929286,6169,100,55000,37,139.762221,35.682194,2,377.18,0.02,55.52,67.23,28.61,13.97,16.38,1.00
3,Antananarivo,1276,0.5,1391433,3097,69,1000,10,47.525600,-18.910000,2,400.54,0.08,0.54,81.54,2.47,6.81,16.29,6.90
4,Nairobi,1795,1.0,4397073,6000,61,1500,12,36.817200,-1.283300,2,881.20,0.40,7.97,54.36,7.15,16.25,39.53,16.72
5,Lima,1540,1.5,9674755,3220,81,4500,15,-77.036526,-12.062107,3,574.11,0.63,44.55,13.95,30.28,27.05,39.31,3.61
